***
#<font color="#274C77" size=10>Challenge TelecomX - PARTE 2</font>
***

## 🧠 Predicción de Cancelación de Clientes (Churn)

Este notebook forma parte del desafío **TelecomX Parte 2**, cuyo objetivo es construir un modelo de Machine Learning para predecir la cancelación de clientes de una empresa ficticia de telecomunicaciones.

**Enlace al Trello del proyecto**: [TelecomX_parte2_Latam](https://trello.com/b/J85wDAhj/telecomxparte2latam)  
**Repositorio GitHub**: [github.com/PamelaOrmeno/TelecomX_parte2_Latam](https://github.com/PamelaOrmeno/TelecomX_parte2_Latam)

---


## 📦 Importación de Librerías

Comenzamos cargando todas las librerías necesarias para manipulación de datos, visualización, preprocesamiento y modelado. Se incluye manejo de errores para asegurar que el entorno tenga todos los paquetes disponibles.

In [ ]:
# Librerías principales
try:
    import pandas as pd
    from pandas import json_normalize
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import plotly.express as px
    print("✅ Librerías de análisis y visualización cargadas correctamente.")
except ImportError as e:
    print(f"❌ Error al importar librerías principales: {e}")

# Librerías para machine learning
try:
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder, StandardScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report, confusion_matrix
    print("✅ Librerías de machine learning cargadas correctamente.")
except ImportError as e:
    print(f"❌ Error al importar librerías de machine learning: {e}")

# Configuraciones generales
try:
    import warnings
    warnings.filterwarnings('ignore')
    pd.set_option('display.max_columns', None)
    print("✅ Configuraciones aplicadas.")
except Exception as e:
    print(f"❌ Error en configuración inicial: {e}")


✅ Librerías de análisis y visualización cargadas correctamente.
✅ Librerías de machine learning cargadas correctamente.
✅ Configuraciones aplicadas.


## 🧱 Carga y Exploración Inicial

En esta etapa cargamos el dataset directamente desde un repositorio público de GitHub. Posteriormente, realizamos una exploración preliminar para entender su estructura general y detectar posibles problemas a resolver.


### 📦 Código Python: Carga desde GitHub

In [ ]:
# Cargar dataset desde GitHub (formato JSON plano)
try:
    import pandas as pd

    url_data = 'https://raw.githubusercontent.com/PamelaOrmeno/TelecomX_parte2_Latam/refs/heads/main/Data/TelecomX_Data.json'
    df = pd.read_json(url_data)
    print(f"✅ Datos cargados desde GitHub. Registros encontrados: {len(df)}")

except Exception as e:
    print(f"❌ Error al cargar datos desde GitHub: {e}")


✅ Datos cargados desde GitHub. Registros encontrados: 7267


### 🔍 Código Python: Exploración Preliminar

In [ ]:
# Exploración general de dimensiones, columnas y tipos de datos
try:
    print("📐 Dimensiones del dataset:", df.shape)

    print("\n🧩 Primeras columnas:")
    display(df.columns.to_frame(index=False).head())

    print("\n🧾 Tipos de datos:")
    display(df.dtypes)

except Exception as e:
    print(f"❌ Error durante la exploración inicial: {e}")


📐 Dimensiones del dataset: (7267, 6)

🧩 Primeras columnas:


,0
0,customerID
1,Churn
2,customer
3,phone
4,internet



🧾 Tipos de datos:


,0
customerID,object
Churn,object
customer,object
phone,object
internet,object
account,object


## 🛠️ Preparación del Dataset para Modelado

En esta sección se realiza el preprocesamiento del dataset. Esto incluye la conversión de variables categóricas a numéricas, el tratamiento de valores faltantes y la normalización de variables numéricas. Estas transformaciones son fundamentales para asegurar que los modelos funcionen correctamente.

---


### 🔤 Codificación de variables categóricas

Convertimos variables categóricas (como "Yes"/"No", "Male"/"Female") a variables numéricas. Se utiliza `LabelEncoder` para variables binarias y `get_dummies()` para aquellas con múltiples categorías.


In [ ]:
# Preparación estructural completa: desanidado, normalización y df_model listo
try:
  # Verificamos que el DataFrame df exista y tenga columnas anidadas
    if 'customer' in df.columns:
        # Desanidamos las columnas estructuradas
        df_customer = json_normalize(df['customer'])
        df_phone = json_normalize(df['phone'])
        df_internet = json_normalize(df['internet'])
        df_account = json_normalize(df['account'])

        # Unimos todo en un nuevo DataFrame plano
        df_flat = pd.concat([
            df['customerID'], df['Churn'],
            df_customer, df_phone, df_internet, df_account
        ], axis=1)

        #print(f"✅ Dataset desanidado correctamente. Dimensiones: {df_flat.shape}")
    else:
        raise ValueError("No se encontraron columnas anidadas ('customer', 'phone', etc.)")

    # Normalizamos nombres de columnas
    df_flat.columns = (
        df_flat.columns
        .str.strip()
        .str.lower()
        .str.replace(' ', '_')
        .str.replace('.', '_')
    )
    #print("✅ Nombres de columnas normalizados.")

    # Eliminamos columnas irrelevantes antes del modelado
    df_model = df_flat.copy()
    if 'customerid' in df_model.columns:
        df_model.drop(columns=['customerid'], inplace=True)
        #print("✅ Columna 'customerid' eliminada del dataset final.")

except Exception as e:
    print(f"❌ Error durante la preparación estructural: {e}")


In [ ]:
df_model.to_excel('archivo_salida.xlsx', sheet_name='Hoja1', index=False)


### ✅ Revisión final del dataset preprocesado

Antes de dividir los datos para el entrenamiento, hacemos una revisión rápida del dataset resultante: número de variables, balance de clases y chequeo de consistencia general.


In [ ]:
# Verificación final antes de modelar
try:
    print("📐 Dimensiones del dataset preprocesado:", df_model.shape)

    print("\n🔍 Tipos de datos:")
    display(df_model.dtypes.value_counts())

    print("\n📊 Balance de clases (variable objetivo):")
    if 'churn' in df_model.columns:
        display(df_model['churn'].value_counts(normalize=True))
    else:
        print("⚠️ No se encontró la columna 'churn' como variable objetivo.")

except Exception as e:
    print(f"❌ Error al revisar dataset final: {e}")


📐 Dimensiones del dataset preprocesado: (7267, 20)

🔍 Tipos de datos:


,count
object,17
int64,2
float64,1



📊 Balance de clases (variable objetivo):


,proportion
churn,
No,0.711986
Yes,0.257190
,0.030824
